It's taken way too long to get to weather, this will only be quite a superficial look into that.

In [1]:
import pandas as pd
import pgaccess as pg

In [2]:
weather = pd.read_csv('../data/delta_weather.csv', parse_dates=['Date time'])

carrierDelay = pd.read_csv('../data/monthly_carrier_delay.csv')
carrierDelay.set_index(['op_unique_carrier', 'month'], inplace=True)

planeSpeed = pd.read_csv('../data/plane_speed.csv')
planeSpeed.set_index(['tail_num'], inplace=True)

ratios = pd.read_csv('../data/airport_delays.csv', index_col=0)
dists = pd.read_csv('../data/distance_delays.csv')
months = pd.read_csv('../data/month_delays.csv')
hours = pd.read_csv('../data/hour_delays.csv')

In [3]:
cities = [
    "Chicago, IL", "Atlanta, GA", "New York, NY", "Dallas/Fort Worth, TX",
    "Denver, CO", "Charlotte, NC", "Los Angeles, CA", "Washington, DC",
    "Houston, TX", "Seattle, WA"
]
sqlfilter = f"""'{"', '".join(cities)}'"""
flightData = pg.get_filtered_test_data(1000000, f'''
    (origin_city_name IN ({sqlfilter}) OR
    dest_city_name IN ({sqlfilter}))
''')
flightData

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,target
0,2018-04-18,DL,DL,DL,1748,DL,N195DN,1748,13487,MSP,...,11292,DEN,"Denver, CO",2011,2122,N,131.0,1.0,680.0,-7.0
1,2018-04-18,DL,DL,DL,1786,DL,N991AT,1786,11433,DTW,...,11057,CLT,"Charlotte, NC",1428,1616,N,108.0,1.0,500.0,-24.0
2,2018-04-18,DL,DL,DL,1789,DL,N822DX,1789,12451,JAX,...,10397,ATL,"Atlanta, GA",1945,2105,N,80.0,1.0,270.0,-8.0
3,2018-04-18,DL,DL,DL,1806,DL,N948AT,1806,14679,SAN,...,14747,SEA,"Seattle, WA",1500,1805,N,185.0,1.0,1050.0,-36.0
4,2018-04-18,DL,DL,DL,1812,DL,N358NB,1812,10397,ATL,...,13930,ORD,"Chicago, IL",2224,2328,N,124.0,1.0,606.0,-12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634737,2018-04-18,DL,DL,DL,1625,DL,N903DE,1625,10397,ATL,...,10821,BWI,"Baltimore, MD",2235,25,N,110.0,1.0,577.0,-1.0
634738,2018-04-18,DL,DL,DL,1640,DL,N591NW,1640,11433,DTW,...,10397,ATL,"Atlanta, GA",600,804,N,124.0,1.0,594.0,-18.0
634739,2018-04-18,DL,DL,DL,1645,DL,N905DA,1645,11433,DTW,...,11292,DEN,"Denver, CO",1215,1324,N,189.0,1.0,1123.0,-2.0
634740,2018-04-18,DL,DL,DL,1666,DL,N922DL,1666,10397,ATL,...,14492,RDU,"Raleigh/Durham, NC",1515,1636,N,81.0,1.0,356.0,-2.0


This block is ported basically directly from Predict Delay 2

Just done as a function to keep it contained. I may move it to another .py file at some point if I actually use it more than once.

In [4]:
def predict2(flightData, carrierDelay, planeSpeed, ratios, dists, months, hours):
    modelData = flightData.copy()
    modelData['month'] = modelData.fl_date.dt.month

    # Monthly carrier delay
    modelData['carrier_delay'] = pd.merge(
        modelData, carrierDelay,
        left_on = ['op_unique_carrier', 'month'],
        right_index = True
    )['mean_delay']

    # Plane speed
    modelData['plane_speed'] = pd.merge(
        modelData, planeSpeed,
        left_on = 'tail_num',
        right_index = True
    )['speed']
    
    modelData = pd.merge(
        pd.merge(modelData, ratios[['origin_ratio']], left_on='origin_airport_id', right_index=True),
        ratios[['dest_ratio']],
        left_on = 'dest_airport_id',
        right_index = True
    )
    modelData = pd.merge(
        pd.merge(modelData, dists),
        months
    )
    
    modelData['hour'] = modelData['crs_arr_time'] // 100
    modelData = pd.merge(modelData, hours)
    
    return modelData

modelData = predict2(flightData, carrierDelay, planeSpeed, ratios, dists, months, hours)
modelData

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,target,month,carrier_delay,plane_speed,origin_ratio,dest_ratio,distance_ratio,month_ratio,hour,hour_ratio
0,2018-04-18,DL,DL,DL,1748,DL,N195DN,1748,13487,MSP,...,-7.0,4,0.163906,0.125235,1.185386e+05,782584.042553,10380.595164,4.474688e+07,21,44313104.0
1,2018-04-30,UA,UA_CODESHARE,UA,5668,OO,N119SY,5668,13487,MSP,...,-10.0,4,3.523944,0.162632,1.185386e+05,782584.042553,10380.595164,4.474688e+07,21,44313104.0
2,2019-04-01,DL,DL,DL,2549,DL,N911DQ,2549,13487,MSP,...,5.0,4,0.163906,0.138221,1.185386e+05,782584.042553,10380.595164,4.474688e+07,21,44313104.0
3,2019-04-21,WN,WN,WN,3188,WN,N744SW,3188,13487,MSP,...,110.0,4,4.800241,0.149995,1.185386e+05,782584.042553,10380.595164,4.474688e+07,21,44313104.0
4,2019-04-28,WN,WN,WN,3188,WN,N7831B,3188,13487,MSP,...,-1.0,4,4.800241,0.150446,1.185386e+05,782584.042553,10380.595164,4.474688e+07,21,44313104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634737,2018-10-27,B6,B6,B6,486,B6,N258JB,486,12478,JFK,...,-10.0,10,3.745263,0.169314,1.840854e+05,62065.425532,20653.998760,3.207822e+07,24,16216.0
634738,2019-02-24,B6,B6,B6,499,B6,N608JB,499,12953,LGA,...,95.0,2,9.964439,0.138503,5.650231e+05,465330.319149,8279.603224,6.279128e+07,24,16216.0
634739,2018-12-24,NK,NK,NK,734,NK,N613NK,734,11298,DFW,...,-16.0,12,2.292346,0.139209,1.484740e+06,231628.191489,8904.339740,5.584268e+07,24,16216.0
634740,2018-12-30,NK,NK,NK,734,NK,N628NK,734,11298,DFW,...,-7.0,12,2.292346,0.139888,1.484740e+06,231628.191489,8904.339740,5.584268e+07,24,16216.0


Merge in the weather

In [5]:
# Just cut off the 'United States'
weather['Name'] = weather['Name'].str[:-len(", United States")]
weather

,Name,Date time,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow,Snow Depth,Wind Speed,Wind Direction,Wind Gust,Visibility,Cloud Cover,Relative Humidity,Conditions
0,"Atlanta, GA",2018-01-01,28.2,18.7,22.3,5.4,NaN,0.00,0.00,0.00,14.6,328.25,25.3,9.9,70.0,51.69,Partially cloudy
1,"Atlanta, GA",2018-01-02,35.1,12.1,23.5,3.1,NaN,0.00,0.00,0.00,6.9,182.33,NaN,9.9,11.6,50.16,Clear
2,"Atlanta, GA",2018-01-03,37.6,26.4,30.7,19.8,NaN,0.00,0.00,0.00,12.0,206.58,18.3,9.9,45.5,36.36,Partially cloudy
3,"Atlanta, GA",2018-01-04,33.6,21.1,26.9,8.5,NaN,0.00,0.00,0.00,20.6,311.58,29.9,9.9,2.7,42.91,Clear
4,"Atlanta, GA",2018-01-05,36.8,16.5,25.6,3.6,NaN,0.00,0.00,0.00,14.2,306.17,23.0,9.9,10.9,40.79,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,"Washington, DC",2020-01-04,58.9,46.9,52.8,40.3,NaN,0.34,0.00,0.00,16.7,222.42,28.7,7.2,79.4,92.45,"Rain, Overcast"
7376,"Washington, DC",2020-01-05,46.0,35.4,40.6,29.5,NaN,0.00,0.00,0.00,21.5,299.08,36.6,9.9,72.3,54.88,Partially cloudy
7377,"Washington, DC",2020-01-06,55.0,35.3,44.1,31.3,NaN,0.00,0.00,0.00,16.9,226.13,27.7,9.9,46.2,49.96,Partially cloudy
7378,"Washington, DC",2020-01-07,45.9,32.9,36.8,25.8,NaN,0.23,0.74,0.74,13.7,167.50,18.3,8.6,66.4,72.52,"Snow, Partially cloudy"


In [6]:
# Only take the interesting columns
interestWeather = weather[['Name', 'Date time', 'Temperature', 'Precipitation', 'Snow', 'Wind Speed', 'Visibility']]
originMerged = pd.merge(
    modelData,
    interestWeather,
    how = 'left',
    left_on = ['origin_city_name', 'fl_date'],
    right_on = ['Name', 'Date time']
)

weatherMerged = pd.merge(
    originMerged,
    interestWeather,
    how = 'left',
    left_on = ['dest_city_name', 'fl_date'],
    right_on = ['Name', 'Date time'],
    suffixes = ('_origin', '_dest')
)
weatherMerged

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,Snow_origin,Wind Speed_origin,Visibility_origin,Name_dest,Date time_dest,Temperature_dest,Precipitation_dest,Snow_dest,Wind Speed_dest,Visibility_dest
0,2018-04-18,DL,DL,DL,1748,DL,N195DN,1748,13487,MSP,...,NaN,NaN,NaN,"Denver, CO",2018-04-18,41.5,0.00,0.0,11.6,9.9
1,2018-04-30,UA,UA_CODESHARE,UA,5668,OO,N119SY,5668,13487,MSP,...,NaN,NaN,NaN,"Denver, CO",2018-04-30,58.0,0.00,0.0,17.1,9.9
2,2019-04-01,DL,DL,DL,2549,DL,N911DQ,2549,13487,MSP,...,NaN,NaN,NaN,"Denver, CO",2019-04-01,43.5,0.00,0.0,17.1,9.9
3,2019-04-21,WN,WN,WN,3188,WN,N744SW,3188,13487,MSP,...,NaN,NaN,NaN,"Denver, CO",2019-04-21,54.5,0.09,0.0,18.6,9.9
4,2019-04-28,WN,WN,WN,3188,WN,N7831B,3188,13487,MSP,...,NaN,NaN,NaN,"Denver, CO",2019-04-28,58.9,0.00,0.0,26.9,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634737,2018-10-27,B6,B6,B6,486,B6,N258JB,486,12478,JFK,...,0.0,29.5,4.3,NaN,NaT,NaN,NaN,NaN,NaN,NaN
634738,2019-02-24,B6,B6,B6,499,B6,N608JB,499,12953,LGA,...,0.0,28.8,5.2,NaN,NaT,NaN,NaN,NaN,NaN,NaN
634739,2018-12-24,NK,NK,NK,734,NK,N613NK,734,11298,DFW,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
634740,2018-12-30,NK,NK,NK,734,NK,N628NK,734,11298,DFW,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


Now drop columns which aren't needed

In [7]:
keepcols = [
    'crs_arr_time', 'distance', 'plane_speed', 'carrier_delay',
    'origin_ratio', 'dest_ratio', 'distance_ratio', 'month_ratio',
    'hour_ratio', 'Temperature_origin', 'Precipitation_origin',
    'Snow_origin', 'Wind Speed_origin', 'Visibility_origin',
    'Temperature_dest', 'Precipitation_dest', 'Snow_dest',
    'Wind Speed_dest', 'Visibility_dest', 'target'
]
modelData = weatherMerged[keepcols].copy()
modelData

,crs_arr_time,distance,plane_speed,carrier_delay,origin_ratio,dest_ratio,distance_ratio,month_ratio,hour_ratio,Temperature_origin,Precipitation_origin,Snow_origin,Wind Speed_origin,Visibility_origin,Temperature_dest,Precipitation_dest,Snow_dest,Wind Speed_dest,Visibility_dest,target
0,2122,680.0,0.125235,0.163906,1.185386e+05,782584.042553,10380.595164,4.474688e+07,44313104.0,NaN,NaN,NaN,NaN,NaN,41.5,0.00,0.0,11.6,9.9,-7.0
1,2103,680.0,0.162632,3.523944,1.185386e+05,782584.042553,10380.595164,4.474688e+07,44313104.0,NaN,NaN,NaN,NaN,NaN,58.0,0.00,0.0,17.1,9.9,-10.0
2,2114,680.0,0.138221,0.163906,1.185386e+05,782584.042553,10380.595164,4.474688e+07,44313104.0,NaN,NaN,NaN,NaN,NaN,43.5,0.00,0.0,17.1,9.9,5.0
3,2105,680.0,0.149995,4.800241,1.185386e+05,782584.042553,10380.595164,4.474688e+07,44313104.0,NaN,NaN,NaN,NaN,NaN,54.5,0.09,0.0,18.6,9.9,110.0
4,2105,680.0,0.150446,4.800241,1.185386e+05,782584.042553,10380.595164,4.474688e+07,44313104.0,NaN,NaN,NaN,NaN,NaN,58.9,0.00,0.0,26.9,9.8,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634737,2400,264.0,0.169314,3.745263,1.840854e+05,62065.425532,20653.998760,3.207822e+07,16216.0,48.0,1.17,0.0,29.5,4.3,NaN,NaN,NaN,NaN,NaN,-10.0
634738,2400,950.0,0.138503,9.964439,5.650231e+05,465330.319149,8279.603224,6.279128e+07,16216.0,42.1,0.50,0.0,28.8,5.2,NaN,NaN,NaN,NaN,NaN,95.0
634739,2400,986.0,0.139209,2.292346,1.484740e+06,231628.191489,8904.339740,5.584268e+07,16216.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-16.0
634740,2400,986.0,0.139888,2.292346,1.484740e+06,231628.191489,8904.339740,5.584268e+07,16216.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.0


Fill in any missing values using the average value in a column. Things like temperature and visibility don't default to 0, and columns that _do_ default to 0 (like precip) will have a significant amount of columns at 0 already to bring the average there.

In [8]:
temp_avg = modelData[['Temperature_origin', 'Temperature_dest']].median().mean() # Whatever, it's close enough
precip_avg = modelData[['Precipitation_origin', 'Precipitation_dest']].median().mean()
snow_avg = modelData[['Snow_origin', 'Snow_dest']].median().mean()
wind_avg = modelData[['Wind Speed_origin', 'Wind Speed_dest']].median().mean()
vis_avg = modelData[['Visibility_origin', 'Visibility_dest']].median().mean()

modelData[['Temperature_origin', 'Temperature_dest']] = \
    modelData[['Temperature_origin', 'Temperature_dest']].fillna(temp_avg)
modelData[['Precipitation_origin', 'Precipitation_dest']] = \
    modelData[['Precipitation_origin', 'Precipitation_dest']].fillna(precip_avg)
modelData[['Snow_origin', 'Snow_dest']] = \
    modelData[['Snow_origin', 'Snow_dest']].fillna(snow_avg)
modelData[['Wind Speed_origin', 'Wind Speed_dest']] = \
    modelData[['Wind Speed_origin', 'Wind Speed_dest']].fillna(wind_avg)
modelData[['Visibility_origin', 'Visibility_dest']] = \
    modelData[['Visibility_origin', 'Visibility_dest']].fillna(vis_avg)

modelData

,crs_arr_time,distance,plane_speed,carrier_delay,origin_ratio,dest_ratio,distance_ratio,month_ratio,hour_ratio,Temperature_origin,Precipitation_origin,Snow_origin,Wind Speed_origin,Visibility_origin,Temperature_dest,Precipitation_dest,Snow_dest,Wind Speed_dest,Visibility_dest,target
0,2122,680.0,0.125235,0.163906,1.185386e+05,782584.042553,10380.595164,4.474688e+07,44313104.0,61.6,0.00,0.0,12.25,9.8,41.5,0.00,0.0,11.60,9.9,-7.0
1,2103,680.0,0.162632,3.523944,1.185386e+05,782584.042553,10380.595164,4.474688e+07,44313104.0,61.6,0.00,0.0,12.25,9.8,58.0,0.00,0.0,17.10,9.9,-10.0
2,2114,680.0,0.138221,0.163906,1.185386e+05,782584.042553,10380.595164,4.474688e+07,44313104.0,61.6,0.00,0.0,12.25,9.8,43.5,0.00,0.0,17.10,9.9,5.0
3,2105,680.0,0.149995,4.800241,1.185386e+05,782584.042553,10380.595164,4.474688e+07,44313104.0,61.6,0.00,0.0,12.25,9.8,54.5,0.09,0.0,18.60,9.9,110.0
4,2105,680.0,0.150446,4.800241,1.185386e+05,782584.042553,10380.595164,4.474688e+07,44313104.0,61.6,0.00,0.0,12.25,9.8,58.9,0.00,0.0,26.90,9.8,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634737,2400,264.0,0.169314,3.745263,1.840854e+05,62065.425532,20653.998760,3.207822e+07,16216.0,48.0,1.17,0.0,29.50,4.3,61.6,0.00,0.0,12.25,9.8,-10.0
634738,2400,950.0,0.138503,9.964439,5.650231e+05,465330.319149,8279.603224,6.279128e+07,16216.0,42.1,0.50,0.0,28.80,5.2,61.6,0.00,0.0,12.25,9.8,95.0
634739,2400,986.0,0.139209,2.292346,1.484740e+06,231628.191489,8904.339740,5.584268e+07,16216.0,61.6,0.00,0.0,12.25,9.8,61.6,0.00,0.0,12.25,9.8,-16.0
634740,2400,986.0,0.139888,2.292346,1.484740e+06,231628.191489,8904.339740,5.584268e+07,16216.0,61.6,0.00,0.0,12.25,9.8,61.6,0.00,0.0,12.25,9.8,-7.0


# Predictions

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

y = modelData.target
X = modelData.drop(columns='target')

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

lr = LinearRegression()
lr.fit(Xtrain, ytrain)
lr.score(Xtest, ytest)

0.04286308836438346

In [10]:
import statsmodels.api as sm

Xconst = sm.add_constant(Xtrain)
testConst = sm.add_constant(Xtest)

model = sm.OLS(ytrain, Xconst)
res = model.fit()

res.summary()

E:\Users\eboyc\anaconda3\envs\bootcamp\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     1099.
Date:                Thu, 23 Sep 2021   Prob (F-statistic):               0.00
Time:                        15:08:26   Log-Likelihood:            -2.5311e+06
No. Observations:              476056   AIC:                         5.062e+06
Df Residuals:                  476036   BIC:                         5.063e+06
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    7.6177      1.545      4.931      0.000       4.590      10.646
crs_arr_time            -0.0007      0.000     -2.374      0.018      -0.001      -0.000
distance                -0.0012      0.000     -7.448      0.000      -0.001      -0.001
plane_speed             18.5793      4.691      3.961      0.000       9.385      27.774
carrier_delay            0.7324      0.018     41.065      0.000       0.697       0.767
origin_ratio           2.11e-06   1.33e-07     15.904      0.000    1.85e-06    2.37e-06
dest_ratio            2.447e-06   1.55e-07     15.786      0.000    2.14e-06    2.75e-06
distance_ratio        6.919e-05   9.16e-06      7.554      0.000    5.12e-05    8.71e-05
month_ratio           3.108e-08   2.95e-09     10.541      0.000    2.53e-08    3.69e-08
hour_ratio            2.889e-07   8.76e-09     32.985      0.000    2.72e-07    3.06e-07
Temperature_origin       0.0596      0.007      8.828      0.000       0.046       0.073
Precipitation_origin    16.3019      0.373     43.698      0.000      15.571      17.033
Snow_origin             11.3579      0.393     28.906      0.000      10.588      12.128
Wind Speed_origin        0.3811      0.022     16.988      0.000       0.337       0.425
Visibility_origin       -1.6852      0.086    -19.696      0.000      -1.853      -1.517
Temperature_dest         0.0341      0.007      5.039      0.000       0.021       0.047
Precipitation_dest      15.2036      0.375     40.553      0.000      14.469      15.938
Snow_dest                9.1635      0.393     23.340      0.000       8.394       9.933
Wind Speed_dest          0.3256      0.022     14.501      0.000       0.282       0.370
Visibility_dest         -1.9279      0.086    -22.544      0.000      -2.095      -1.760
==============================================================================
Omnibus:                   702870.649   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        516833742.994
Skew:                           8.834   Prob(JB):                         0.00
Kurtosis:                     163.448   Cond. No.                     4.70e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.7e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
import xgboost as xgb
from sklearn.metrics import r2_score

xgreg = xgb.XGBRegressor(
    max_depth = 5,
    n_estimators = 50
)
xgreg.fit(Xtrain, ytrain)
pred = xgreg.predict(Xtest)

r2_score(ytest, pred)

0.07824273605966958

In [12]:
from pickler import write_model
write_model('../models/xgb-weather', xgreg)